In [1]:
import matplotlib.pyplot as plt
import os
import random
import time

import pyBeamSim
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from model.model_with_attention import Encoder_Decoder

In [2]:
df = pd.read_csv('../data/data_50000.csv', index_col=0)
df = df[:10000]
df

,drift1_len,quad1_len,quad1_gra,drift2_len,quad2_len,quad2_gra,x_avg0,x_avg1,x_avg2,x_avg3,x_avg4
0,0.196427,0.182149,8.681705,0.250776,0.373205,-10.382813,-2.582750e-09,0.000022,-5.354477e-07,-0.000005,0.000004
1,0.403793,0.429330,9.532400,0.329042,0.112563,5.724299,-2.582750e-09,0.000042,5.218342e-07,0.000214,-0.000167
2,0.329227,0.152616,17.639344,0.157597,0.423441,-10.074239,-2.582750e-09,0.000036,-1.753044e-06,0.000002,0.000137
3,0.301287,0.423994,4.371640,0.204886,0.154771,15.747145,-2.582750e-09,0.000034,1.601237e-07,0.000001,-0.000005
4,0.349911,0.361642,-5.058993,0.480674,0.409044,15.117262,-2.582750e-09,0.000039,-2.395925e-06,-0.000260,0.000038
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.396447,0.382212,5.604220,0.354495,0.222928,-15.905318,-2.582750e-09,0.000043,1.511706e-07,-0.000011,0.000103
9996,0.114760,0.346819,-6.901604,0.369893,0.397082,-7.834693,-2.582750e-09,0.000013,-2.018009e-06,-0.000080,0.000078
9997,0.490315,0.249131,-1.215864,0.268813,0.141644,-13.663927,-2.582750e-09,0.000044,-2.898933e-07,-0.000019,-0.000071
9998,0.342286,0.275773,3.625842,0.436554,0.132755,-1.138420,-2.582750e-09,0.000038,-1.473266e-06,-0.000029,-0.000023


In [3]:
scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
scaler1.fit_transform(df)
epsilon = 1e-10

change_columns = df.columns[-4:].tolist()
# change_columns.remove('x_avg0')
df[change_columns] = scaler2.fit_transform(df[change_columns])
df[change_columns] = np.where(df[change_columns] == 0, epsilon, df[change_columns])
# df['x_avg0'] = 0
df

,drift1_len,quad1_len,quad1_gra,drift2_len,quad2_len,quad2_gra,x_avg0,x_avg1,x_avg2,x_avg3,x_avg4
0,0.196427,0.182149,8.681705,0.250776,0.373205,-10.382813,-2.582750e-09,-0.916464,0.037908,0.325505,0.351625
1,0.403793,0.429330,9.532400,0.329042,0.112563,5.724299,-2.582750e-09,0.974384,0.519220,1.203486,-0.791223
2,0.329227,0.152616,17.639344,0.157597,0.423441,-10.074239,-2.582750e-09,0.406076,-0.516384,0.351111,1.238314
3,0.301287,0.423994,4.371640,0.204886,0.154771,15.747145,-2.582750e-09,0.181292,0.354557,0.348436,0.293746
4,0.349911,0.361642,-5.058993,0.480674,0.409044,15.117262,-2.582750e-09,0.665285,-0.809047,-0.698823,0.578823
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.396447,0.382212,5.604220,0.354495,0.222928,-15.905318,-2.582750e-09,1.005996,0.350481,0.299260,1.014616
9996,0.114760,0.346819,-6.901604,0.369893,0.397082,-7.834693,-2.582750e-09,-1.790811,-0.637006,0.021067,0.848611
9997,0.490315,0.249131,-1.215864,0.268813,0.141644,-13.663927,-2.582750e-09,1.149614,0.149693,0.267634,-0.147845
9998,0.342286,0.275773,3.625842,0.436554,0.132755,-1.138420,-2.582750e-09,0.590942,-0.389020,0.225882,0.168832


In [4]:
input_size = 1
output_size = 1
use_models = ['GRU', 'LSTM']

batch_size = 64
hidden_size = 64
num_layers = 2
dropout = 0
weight_decay = 0
teacher_forcing = 0.5

torch.backends.cudnn.benchmark = True
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
use_model = use_models[0]

# last 5 columns as y_true
X = df.iloc[:, :-5]
y = df.iloc[:, -5:]
# transform df into tensor
X = torch.Tensor(X.values).unsqueeze(2) # add a dim: input_size
y = torch.Tensor(y.values).unsqueeze(2)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

print(f'y_train.shape: {y_train.shape}')
print(f'X_train.shape: {X_train.shape}')

y_train.shape: torch.Size([8000, 5, 1])
X_train.shape: torch.Size([8000, 6, 1])


In [5]:
from torch.utils.data import TensorDataset, DataLoader

# generate dataset and dataloader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

first_y = train_dataset[0][1]
output_len = first_y.shape[0] - 1
print('the first y: ', first_y)

the first y:  tensor([[-2.5827e-09],
        [-6.6649e-01],
        [-2.2531e+00],
        [-1.0563e-01],
        [-5.2295e-01]])


### functions

In [6]:
# use mean absolute percent error as criterion
# get mean mape for each batch
def get_mape(tensor_true, tensor_pred):
    """
    :param tensor_true : (batch_size, output_size) or (output_size,)
    :param tensor_pred : same shape as tensor_true
    """

    mape = torch.abs((tensor_pred - tensor_true) / tensor_true) * 100

    if tensor_true.dim() == 2:
        return torch.mean(mape, dim=1).mean()  
    else:
        return torch.mean(mape)  
# and R2 score

def compute_column_mape(tensor_true, tensor_pred):
    mape = torch.abs((tensor_pred - tensor_true) / tensor_true) * 100
    mape_per_column = torch.mean(mape, dim=0)  # for column
    return mape_per_column

def count_all_parameters(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

y_true = torch.Tensor([100, 200, 300])
y_pred = torch.Tensor([110, 190, 310])
r2 = r2_score(y_true, y_pred)
mape = get_mape(y_true, y_pred)
print(f'R^2 Score: {r2:.2f}')
print(f'MAPE: {mape:.2f}%')

R^2 Score: 0.98
MAPE: 6.11%


In [7]:
def train(model, dataLoader, test_loader, optimizer, criterion, device, num_epochs, clip=1):
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
    train_losses = []
    test_losses = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X, y in dataLoader:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_pred = model(X, y, use_tf=True)
            # remove the first avg_0(the input)
            loss = criterion(y_pred, y[:, 1:, :])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            total_loss += loss.item() # for a batch
        
        avg_train_loss = total_loss / len(dataLoader)   # len(dataloader): nums of batch
        train_losses.append(avg_train_loss)
        
        model.eval()
        total_val_loss = 0
        with torch.no_grad():  
            for X_val, y_val in test_loader:
                X_val = X_val.to(device)
                y_val = y_val.to(device)
                
                y_val_pred = model(X_val, y_val)
                val_loss = criterion(y_val_pred, y_val[:, 1:, :])
                total_val_loss += val_loss.item()
        
        avg_val_loss = total_val_loss / len(test_loader)
        test_losses.append(avg_val_loss)
        
        # scheduler.step()
        
        print(f'Epoch {epoch+1}: train_loss: {avg_train_loss:.10f}, valid_loss: {avg_val_loss:.10f}')
    return train_losses, test_losses
        
def evaluate(model, dataLoader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X, y in dataLoader:
            X,y = X.to(device), y.to(device)
            y_pred = model(X, y)
            loss = criterion(y[:, 1:, :], y_pred)
            total_loss += loss.item()
    return total_loss

def evaluate_test_mape_gpu(model, test_loader):
    model.eval()
    total_num = 0
    total_mape = 0
    with torch.no_grad():
        for X, y in test_loader:
            X = X.to('cuda')
            y = y.to('cuda')
            y_pred = model(X, y)
            mape = get_mape(y[:, 1:, :], y_pred)
            total_mape += mape
            total_num += X.shape[0]
    return total_mape / len(test_loader)

def evaluate_test_mape_gpu_column(model, test_loader):
    model.eval()
    total_mape = None  

    with torch.no_grad():
        for X, y in test_loader:
            X = X.to('cuda')
            y = y.to('cuda')
            y_pred = model(X, y)

            mape = compute_column_mape(y[:, 1:, :], y_pred)  
            if total_mape is None:
                total_mape = mape
            else:
                total_mape += mape
    
    total_mape = total_mape.squeeze().tolist()
    column_mape = [round(per_mape / len(test_loader), 2) for per_mape in total_mape]
    return column_mape

### train

In [8]:
model = Encoder_Decoder(input_size=input_size,
                        encoder_hidden_size=hidden_size,
                        decoder_hidden_size=hidden_size,
                        output_size=output_size,
                        model_device=device,
                        num_layers=num_layers,
                        teacher_forcing=teacher_forcing,
                        dropout=dropout)

model.to(device)
print(f'model has {count_all_parameters(model)} parameters')
# model

model has 191617 parameters


In [9]:
num_epochs = 400
learning_rate = 0.001

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.MSELoss()

train_losses, test_losses = train(model, train_loader, test_loader, optimizer, criterion, device, num_epochs)


Epoch 1: train_loss: 0.9776477370, valid_loss: 0.9915959984
Epoch 2: train_loss: 0.9366370344, valid_loss: 0.9201309532
Epoch 3: train_loss: 0.6897199731, valid_loss: 0.6529634530
Epoch 4: train_loss: 0.5711501818, valid_loss: 0.6155170519
Epoch 5: train_loss: 0.5129190326, valid_loss: 0.5076123867
Epoch 6: train_loss: 0.4676607955, valid_loss: 0.4952647947
Epoch 7: train_loss: 0.4388744477, valid_loss: 0.5232056775
Epoch 8: train_loss: 0.4331955042, valid_loss: 0.4336161716
Epoch 9: train_loss: 0.3840420827, valid_loss: 0.4076882638
Epoch 10: train_loss: 0.3747138909, valid_loss: 0.3755833241
Epoch 11: train_loss: 0.3534379152, valid_loss: 0.3848207705
Epoch 12: train_loss: 0.3334836854, valid_loss: 0.4194956333
Epoch 13: train_loss: 0.3223312169, valid_loss: 0.4023948731
Epoch 14: train_loss: 0.3016265835, valid_loss: 0.3397919331
Epoch 15: train_loss: 0.2962967891, valid_loss: 0.3192794067
Epoch 16: train_loss: 0.2900438592, valid_loss: 0.3242895678
Epoch 17: train_loss: 0.274068263

### save model and loss

In [10]:
save_pth = '../model_save'
model_pth = 'seq2seq' + '_' + str(num_layers) + 'layers' +'.pth'
torch.save(model.state_dict(), os.path.join(save_pth, model_pth))

In [11]:
save_pth2 = '../record_for_plot'
df_record = pd.DataFrame({
    'epoch': range(1, num_epochs + 1),
    'train_loss': train_losses,
    'test_loss': test_losses
})
loss_pth = 'seq2seq' + '_' + str(num_layers) + 'layers' +'.csv'
df_record.to_csv(os.path.join(save_pth2, loss_pth), index=False)

### result analysis

In [12]:
model.load_state_dict(torch.load('../model_save/seq2seq_2layers.pth'))

# result analysis
test_mape = evaluate_test_mape_gpu(model, test_loader)
train_mape = evaluate_test_mape_gpu(model, train_loader)
test_column_mape = evaluate_test_mape_gpu_column(model, test_loader)
train_column_mape = evaluate_test_mape_gpu_column(model, train_loader)
print(f'test_mape: {test_mape:.4f}')
print(f'train_mape: {train_mape:.4f}')
print(f'for each column \ntest_column_mape: {test_column_mape} \ntrain_column_mape: {train_column_mape}')

test_mape: 186.0795
train_mape: 85.5267
for each column 
test_column_mape: [37.79, 203.04, 157.05, 346.44] 
train_column_mape: [42.34, 178.95, 55.74, 65.07]


### result record

In [13]:
"""
MAPE and loss

2 layers without dropout, weight_decay, scheduler, lr=0.001
for 1000:
Epoch 250: train_loss: 0.0038760487, valid_loss: 0.6360880136
test: 254.8839
train: 46.2822   , 25.9250 if all columns standard

for 10000:
Epoch 400: train_loss: 0.0033661677, valid_loss: 0.2318614242
test: 186.9136
train: 54.1513

layer3
Epoch 250: train_loss: 0.0079129066, valid_loss: 0.2666302446
test: 273.7292
train: 64.5897

4 layers 
Epoch 250: train_loss: 0.0151903867, valid_loss: 0.2816597309
test: 374.9942
train: 71.7741
"""
print('daijiaobu')

daijiaobu


### predict once using random data from dataset


In [14]:
def print_tensor(a: torch.Tensor, is_pred=True):
    list = a.squeeze().numpy().tolist()
    c = [round(b, 2) for b in list]
    if is_pred:
        print(c)
    else:
        print(c[1:])
    
model.to('cpu')
num_train = random.randint(0, X_train.shape[0])
num_test = random.randint(0, X_test.shape[0])

train_input = train_dataset[num_train][0].reshape(1, -1, 1)
train_output = train_dataset[num_train][1].reshape(1, -1, 1)
test_input = test_dataset[num_test][0].reshape(1, -1, 1)
test_output = test_dataset[num_test][1].reshape(1, -1, 1)

model.eval()
with torch.no_grad():
    start_time = time.time()
    train_pred = model(train_input, train_output)
    end_time = time.time()
    test_pred = model(test_input, test_output)

consume_time = end_time - start_time
print(f'one predict time: {consume_time:.4f} seconds\n')
print('choose from train:')
print_tensor(train_output, False)
print_tensor(train_pred)
print(f'train_mape: {get_mape(train_output[:, 1:, :], train_pred)}\n')

print('choose from test:')
print_tensor(test_output, False)
print_tensor(test_pred)
print(f'test_mape: {get_mape(test_output[:, 1:, :], test_pred)}')

one predict time: 0.0231 seconds

choose from train:
[0.6, 0.54, 0.33, -0.28]
[0.5, 0.47, 0.24, -0.27]
train_mape: 15.42165470123291

choose from test:
[0.18, 0.39, 0.42, -0.2]
[0.14, 0.86, 0.7, 0.02]
test_mape: 79.69793701171875


### generalization